In [ ]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

# Window Notifier Testing

This notebook demonstrates the WindowNotifier functionality by simulating various window coverage gap scenarios.

**SNS Topic ARN:** `arn:aws:sns:eu-west-1:760097843905:testing-alerting-chatbot-events`

**Note:** Messages are threaded by 6-hour chunks to prevent spam. Messages within the same chunk will appear in the same thread.

In [ ]:
from datetime import datetime, timezone, timedelta

import boto3
from clients.chatbot_notifier import ChatbotNotifier
from adapters.utils.window_notifier import WindowNotifier
from adapters.utils.window_reporter import WindowCoverageReport, CoverageGap

# Configure SNS client
sns_client = boto3.client("sns", region_name="eu-west-1")
topic_arn = "arn:aws:sns:eu-west-1:760097843905:testing-alerting-chatbot-events"

# Create the chatbot notifier
chatbot_notifier = ChatbotNotifier(sns_client=sns_client, topic_arn=topic_arn)

# Create the window notifier
window_notifier = WindowNotifier(
    chatbot_notifier=chatbot_notifier,
    table_name="axiell_window_status.window_status",
)

print("✅ WindowNotifier initialized successfully")

## Scenario 1: Single Coverage Gap

A simple case with one missing window of 2 hours.

In [ ]:
# Create a report with a single 2-hour gap
now = datetime.now(timezone.utc)
range_start = now - timedelta(hours=24)
range_end = now

single_gap_report = WindowCoverageReport(
    range_start=range_start,
    range_end=range_end,
    coverage_gaps=[
        CoverageGap(
            start=now - timedelta(hours=5),
            end=now - timedelta(hours=3),
        )
    ],
    total_windows=10,
    last_success_end=now - timedelta(hours=6),
)

# Send notification
window_notifier.notify_if_gaps(
    report=single_gap_report,
    job_id="20251202T1200",
    trigger_time=now,
)

print(f"✅ Sent notification for single gap at {now.strftime('%H:%M:%S')}")

## Scenario 2: Multiple Small Gaps (Under Display Limit)

Three gaps totaling 4.5 hours - all will be displayed.

In [ ]:
# Create a report with 3 gaps
now = datetime.now(timezone.utc)
range_start = now - timedelta(hours=24)
range_end = now

multiple_gaps_report = WindowCoverageReport(
    range_start=range_start,
    range_end=range_end,
    coverage_gaps=[
        CoverageGap(
            start=now - timedelta(hours=10),
            end=now - timedelta(hours=8),  # 2 hours
        ),
        CoverageGap(
            start=now - timedelta(hours=6),
            end=now - timedelta(hours=4.5),  # 1.5 hours
        ),
        CoverageGap(
            start=now - timedelta(hours=2),
            end=now - timedelta(hours=1),  # 1 hour
        ),
    ],
    total_windows=20,
    last_success_end=now - timedelta(hours=12),
)

# Send notification
window_notifier.notify_if_gaps(
    report=multiple_gaps_report,
    job_id="20251202T1230",
    trigger_time=now,
)

print(f"✅ Sent notification for 3 gaps at {now.strftime('%H:%M:%S')}")

## Scenario 3: Many Gaps (Exceeds Display Limit)

8 gaps totaling 12 hours - only first 5 will be shown with summary of remaining.

In [ ]:
# Create a report with 8 gaps (exceeds MAX_GAPS_TO_DISPLAY of 5)
now = datetime.now(timezone.utc)
base_time = now - timedelta(hours=24)
range_start = base_time - timedelta(hours=2)
range_end = now

many_gaps_report = WindowCoverageReport(
    range_start=range_start,
    range_end=range_end,
    coverage_gaps=[
        CoverageGap(
            start=base_time + timedelta(hours=i * 2.5),
            end=base_time + timedelta(hours=i * 2.5 + 1.5),  # 1.5 hours each
        )
        for i in range(8)
    ],
    total_windows=50,
    last_success_end=base_time - timedelta(hours=2),
)

# Send notification
window_notifier.notify_if_gaps(
    report=many_gaps_report,
    job_id="20251202T1300",
    trigger_time=now,
)

print(f"✅ Sent notification for 8 gaps at {now.strftime('%H:%M:%S')}")
print(f"   (Only first 5 displayed + summary of remaining 3)")

## Scenario 4: Messages Within Same 6-Hour Chunk (Threading Test)

Send two messages within the same 6-hour chunk - they should appear in the same thread.

The day is divided into chunks:
- 00:00-06:00 (chunk 0)
- 06:00-12:00 (chunk 1)
- 12:00-18:00 (chunk 2)
- 18:00-24:00 (chunk 3)

In [ ]:
# First message in the chunk
now = datetime.now(timezone.utc)
range_start = now - timedelta(hours=12)
range_end = now

chunk_test_report_1 = WindowCoverageReport(
    range_start=range_start,
    range_end=range_end,
    coverage_gaps=[
        CoverageGap(
            start=now - timedelta(hours=3),
            end=now - timedelta(hours=2),
        )
    ],
    total_windows=15,
    last_success_end=now - timedelta(hours=4),
)

window_notifier.notify_if_gaps(
    report=chunk_test_report_1,
    job_id="20251202T1330",
    trigger_time=now,
)

print(f"✅ Sent first message at {now.strftime('%H:%M:%S')} (chunk {now.hour // 6})")

# Wait a moment (simulate time passing)
import time
time.sleep(2)

# Second message in the same chunk (a few seconds later)
now2 = datetime.now(timezone.utc)
range_start2 = now2 - timedelta(hours=12)
range_end2 = now2

chunk_test_report_2 = WindowCoverageReport(
    range_start=range_start2,
    range_end=range_end2,
    coverage_gaps=[
        CoverageGap(
            start=now2 - timedelta(hours=2.5),
            end=now2 - timedelta(hours=1.5),
        )
    ],
    total_windows=15,
    last_success_end=now2 - timedelta(hours=3),
)

window_notifier.notify_if_gaps(
    report=chunk_test_report_2,
    job_id="20251202T1331",
    trigger_time=now2,
)

print(f"✅ Sent second message at {now2.strftime('%H:%M:%S')} (chunk {now2.hour // 6})")
print(f"   Both messages should appear in the same thread!")

## Scenario 5: Simulated Messages Across Different Chunks

Create messages with timestamps in different 6-hour chunks to test thread separation.

In [ ]:
# Create timestamps in different chunks
today = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

# Chunk 0: 02:00 (00:00-06:00)
time_chunk0 = today.replace(hour=2, minute=30)

# Chunk 1: 08:00 (06:00-12:00)
time_chunk1 = today.replace(hour=8, minute=30)

# Chunk 2: 14:00 (12:00-18:00)
time_chunk2 = today.replace(hour=14, minute=30)

# Chunk 3: 20:00 (18:00-24:00)
time_chunk3 = today.replace(hour=20, minute=30)

# Send messages for each chunk
for i, trigger_time in enumerate([time_chunk0, time_chunk1, time_chunk2, time_chunk3]):
    range_start = trigger_time - timedelta(hours=12)
    range_end = trigger_time
    
    chunk_report = WindowCoverageReport(
        range_start=range_start,
        range_end=range_end,
        coverage_gaps=[
            CoverageGap(
                start=trigger_time - timedelta(hours=2),
                end=trigger_time - timedelta(hours=1),
            )
        ],
        total_windows=12,
        last_success_end=trigger_time - timedelta(hours=3),
    )
    
    window_notifier.notify_if_gaps(
        report=chunk_report,
        job_id=f"20251202T{trigger_time.hour:02d}00",
        trigger_time=trigger_time,
    )
    
    chunk_num = trigger_time.hour // 6
    print(f"✅ Sent message for chunk {chunk_num} at {trigger_time.strftime('%H:%M')}")

print("\n🧵 Each message should be in a different thread (4 separate threads)")

## Scenario 6: Large Gap with Historical Context

A report with a significant gap showing the last successful harvest was days ago.

In [ ]:
# Create a report with a large 48-hour gap
now = datetime.now(timezone.utc)
range_start = now - timedelta(hours=96)  # 4 days back
range_end = now

large_gap_report = WindowCoverageReport(
    range_start=range_start,
    range_end=range_end,
    coverage_gaps=[
        CoverageGap(
            start=now - timedelta(hours=72),  # Started 3 days ago
            end=now - timedelta(hours=24),    # Ended 1 day ago (48 hour gap)
        )
    ],
    total_windows=100,
    last_success_end=now - timedelta(hours=73),  # Last success was 3+ days ago
)

# Send notification
window_notifier.notify_if_gaps(
    report=large_gap_report,
    job_id="20251202T1400",
    trigger_time=now,
)

print(f"✅ Sent notification for 48-hour gap")
print(f"   Gap: {(large_gap_report.coverage_gaps[0].end - large_gap_report.coverage_gaps[0].start).total_seconds() / 3600:.1f} hours")

## Scenario 7: No Gaps (No Notification)

Test that nothing is sent when there are no gaps.

In [ ]:
# Create a report with no gaps
now = datetime.now(timezone.utc)
range_start = now - timedelta(hours=24)
range_end = now

no_gaps_report = WindowCoverageReport(
    range_start=range_start,
    range_end=range_end,
    coverage_gaps=[],  # Empty list = no gaps
    total_windows=25,
    last_success_end=now - timedelta(hours=1),
)

# Try to send notification (should be silently skipped)
window_notifier.notify_if_gaps(
    report=no_gaps_report,
    job_id="20251202T1430",
    trigger_time=now,
)

print("✅ No notification sent (no gaps detected) - this is expected!")